In [116]:
import pandas as pd
import json 

In [117]:
train = pd.read_csv('./data/train.tsv', sep='\t', header=None, names=['text', 'emotion_ids', 'id'])

In [118]:
# train

In [119]:
emotions = pd.read_csv('./data/emotions.txt', header=None, names=['emotion'])
# emotions

In [120]:
def transform_emotion_ids(emotion_ids):
    emotion_ids_list = emotion_ids.split(",")
    
    emotion_names = []
    for idx in emotion_ids_list:
        emotion_names.append(emotions.iloc[int(idx)][0])
        
    return emotion_names

In [121]:
train['emotions'] = train['emotion_ids'].apply(transform_emotion_ids)

In [122]:
# train.iloc[20:]

In [123]:
with open("./data/ekman_mapping.json", "r") as read_file:
    ekman_mapping = json.load(read_file)

In [124]:
# ekman_mapping

In [125]:
def ekman_emotions(emotion_list):
    ekman = []
    for emotion in emotion_list:
        if emotion != 'neutral':
            key = [k for k, v in ekman_mapping.items() if emotion in v]
            if key[0] not in ekman:
                ekman.append(key[0])
        else:
            ekman.append('neutral')
    return ekman

In [126]:
train['ekman_emotions'] = train['emotions'].apply(ekman_emotions)

In [127]:
train

,text,emotion_ids,id,emotions,ekman_emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[neutral],[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[anger],[anger]
3,To make her feel threatened,14,ed7ypvh,[fear],[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[annoyance],[anger]
...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738,[love],[joy]
43406,Always thought that was funny but is it a refe...,6,ee7fdou,[confusion],[surprise]
43407,What are you talking about? Anything bad that ...,3,efgbhks,[annoyance],[anger]
43408,"More like a baptism, with sexy results!",13,ed1naf8,[excitement],[joy]


In [128]:
emotions_ekman = pd.read_csv('./data/ekman_emotions.txt', header=None, names=['emotion'])
emotions_ekman

,emotion
0,anger
1,disgust
2,fear
3,joy
4,sadness
5,surprise
6,neutral


In [129]:
def ekman_ids(ekman_list):
    ekman = []
    for x in ekman_list:
        idx = np.where(emotions_ekman['emotion']==x)[0][0]
        ekman.append(int(idx))
    return ekman 

In [130]:
train['ekman_ids'] = train['ekman_emotions'].apply(ekman_ids)

In [131]:
train

,text,emotion_ids,id,emotions,ekman_emotions,ekman_ids
0,My favourite food is anything I didn't have to...,27,eebbqej,[neutral],[neutral],[6]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[neutral],[neutral],[6]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[anger],[anger],[0]
3,To make her feel threatened,14,ed7ypvh,[fear],[fear],[2]
4,Dirty Southern Wankers,3,ed0bdzj,[annoyance],[anger],[0]
...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738,[love],[joy],[3]
43406,Always thought that was funny but is it a refe...,6,ee7fdou,[confusion],[surprise],[5]
43407,What are you talking about? Anything bad that ...,3,efgbhks,[annoyance],[anger],[0]
43408,"More like a baptism, with sexy results!",13,ed1naf8,[excitement],[joy],[3]


In [133]:
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in tqdm(range(len(df))):
        temp = [0] * 7
        label_indices = df.iloc[i]["ekman_ids"]
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)
        
    return pd.DataFrame(one_hot_encoding)

train_ohe_labels = one_hot_encoder(train)
# valid_ohe_labels = one_hot_encoder(valid)
# test_ohe_labels = one_hot_encoder(test)

print(train_ohe_labels.shape)
#(43410, 28)

train = pd.concat([train, train_ohe_labels], axis=1)
# valid = pd.concat([valid, valid_ohe_labels], axis=1)
# test = pd.concat([test, test_ohe_labels], axis=1)

  0%|          | 0/43410 [00:00<?, ?it/s]

(43410, 7)


In [134]:
train

,text,emotion_ids,id,emotions,ekman_emotions,ekman_ids,0,1,2,3,4,5,6
0,My favourite food is anything I didn't have to...,27,eebbqej,[neutral],[neutral],[6],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[neutral],[neutral],[6],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[anger],[anger],[0],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[fear],[fear],[2],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[annoyance],[anger],[0],1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738,[love],[joy],[3],0,0,0,1,0,0,0
43406,Always thought that was funny but is it a refe...,6,ee7fdou,[confusion],[surprise],[5],0,0,0,0,0,1,0
43407,What are you talking about? Anything bad that ...,3,efgbhks,[annoyance],[anger],[0],1,0,0,0,0,0,0
43408,"More like a baptism, with sexy results!",13,ed1naf8,[excitement],[joy],[3],0,0,0,1,0,0,0
